In [1]:
#### Standard Libraries ####
import os
from pprint import pprint
import numpy as np
import pandas as pd
import multiprocessing as mp
from functools import partial
from itertools import product
import timeit
import uuid

#### third-party Libraries ####
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier as SRC
from lolopy.learners import RandomForestClassifier as LRC
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from matminer.featurizers.base import MultipleFeaturizer

#### Local Libraries ####
from utils import (Result, run_k_folds, 
                   report_column_labels,
                   compile_data)
from data_manager import DataManager
from featurizer import Featurizer

In [2]:
# configuration
np.random.seed(8)
load_path = os.path.join('data','training_data.csv')
save_path = os.path.join('results','test_baseline.csv')
mp_api_key = '7n6DwPUQ5cf8ZTWO'
oversample = False
data_ramp = False
feature_set = ['energy']

In [10]:
# Load Data
dm = DataManager(load_path, save_path)
dm.load()

'Loaded 2572 records.'


In [12]:
f = Featurizer(feature_set)

In [13]:
dm.featurized_data = f.featurize(dm.data)

In [16]:
tdf = pd.DataFrame(dm.featurized_data)

In [19]:
dm.data

,formula,composition,group,stable
0,Cs,(Cs),1,1
1,K,(K),2,1
2,Ba,(Ba),3,1
3,Sr,(Sr),4,1
4,Br,(Br),5,1
5,Na,(Na),6,1
6,La,(La),7,1
7,Pr,(Pr),8,1
8,Ce,(Ce),9,1
9,Te,(Te),10,1


In [11]:
# Format and careate composition objects
#dm.compute_formula()
dm.to_binary_classes()
dm.get_pymatgen_composition()
dm.remove_noble_gasses()
dm.remove_features()

In [11]:
# Test featurizing a single compound

In [3]:
test_df = pd.DataFrame(['Na'], columns=['formula'])

In [4]:
dm.data = test_df

NameError: name 'dm' is not defined

In [207]:
# convert to pmg composition object
dm.get_pymatgen_composition()

In [211]:
dm.data.iloc[0,1].as_dict()

defaultdict(float, {'Na': 0.5, 'Cl': 0.5})

In [187]:
from matminer.featurizers import composition as cf
features = [cf.CohesiveEnergy(mapi_key=mp_api_key)]
f = MultipleFeaturizer(features)

In [188]:
dm.featurized_data = np.array(f.featurize_many(dm.data['composition'], ignore_errors=True))


In [189]:
# For a single element it works
dm.featurized_data

array([[3.36142633]])

In [191]:
sto_df = pd.read_csv('data/sto_table.csv')

In [192]:
sto_df

,frac_a,frac_b,sto_a,sto_b
0,0.1,0.9,1,9
1,0.2,0.8,1,4
2,0.3,0.7,1,2
3,0.4,0.6,2,3
4,0.5,0.5,1,1
5,0.6,0.4,3,2
6,0.7,0.3,2,1
7,0.8,0.2,4,1
8,0.9,0.1,9,1


In [ ]:
def get_sto_formula(row):
    comp = row['composition'].as_dict()
    sto = [val for key, val in comp]
    
    

In [8]:
round(0.3000,1)

0.3